<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
# most of this code is from the tensorflow tutorial


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import time
from scipy import stats 

tf.enable_eager_execution()

In [3]:
# setting up mnist data

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [6]:
# high-level Keras model

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

11493376/11490434 [==============================] - 56s 5us/step


W0619 13:07:04.153959 4345886144 deprecation.py:323] From /Users/lsorense/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
60000/60000 [==============================] - 8s 130us/sample - loss: 0.2216 - acc: 0.9346
Epoch 2/5
60000/60000 [==============================] - 8s 127us/sample - loss: 0.0949 - acc: 0.9711
Epoch 3/5
60000/60000 [==============================] - 7s 123us/sample - loss: 0.0690 - acc: 0.9784
Epoch 4/5
60000/60000 [==============================] - 5s 91us/sample - loss: 0.0535 - acc: 0.9827
Epoch 5/5
10000/10000 [==============================] - 0s 43us/sample - loss: 0.0581 - acc: 0.9821


[0.0581096921046963, 0.9821]

In [ ]:
# high-level Keras model in loop

num_models = 5
models = []

for i in range(num_models):
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(512, activation=tf.nn.relu),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=5)
    models.append(model)

In [51]:
def evaluate_ensemble(model_list, x_test, y_test):
    preds_list = [] 
    
    for model in model_list:
        preds = model.predict(x_test)
        preds_list.append(preds)
    
    ensemble_pred = np.mean(np.stack(preds_list),axis=0)

    label_pred = tf.math.argmax(ensemble_pred,axis=1)
    
    loss = tf.losses.softmax_cross_entropy(tf.one_hot(y_test,10), ensemble_pred)
    print("Loss: ", loss)
    
    accuracy = 1 - (np.sum(y_test == label_pred)/10000.0)
    print("Accuracy: ", accuracy)
    
    return ensemble_pred
    

evaluate_ensemble(models, x_test, y_test)

Loss:  tf.Tensor(1.4850203, shape=(), dtype=float32)
Accuracy:  1.0


array([[1.9896795e-09, 2.4574625e-09, 1.5480719e-08, ..., 9.9998075e-01,
        9.6531094e-09, 2.8743189e-06],
       [2.8822966e-09, 1.7037626e-06, 9.9999809e-01, ..., 4.4510084e-14,
        6.5817190e-08, 4.4665001e-13],
       [1.6269691e-08, 9.9991024e-01, 1.6426715e-05, ..., 4.6757901e-05,
        1.5032619e-05, 8.1201776e-08],
       ...,
       [1.0836306e-13, 1.7991258e-09, 3.0248940e-13, ..., 3.5904482e-06,
        6.2107951e-08, 4.9836159e-05],
       [2.5203140e-09, 2.8284572e-10, 5.7193634e-12, ..., 5.8851000e-09,
        9.3742536e-04, 1.8913093e-11],
       [1.0471130e-09, 1.4950366e-11, 2.4768034e-09, ..., 6.6273023e-14,
        8.4541718e-10, 5.9797245e-12]], dtype=float32)

In [ ]:
# Keras model with modified optimizer

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)